# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [1]:
var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var writeFileSync = require('fs').writeFileSync;

// add some run commands to the bash script
var bashToRun = (code) => code.split('\n').map(l => 'RUN ' + l)
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');

// create a selenium Dockerfile with a vnc connection
var interpretAllPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');
var interpret, interpretObject;
var seleniumDocker = (outputFile) => {
    return importer.importNotebook(interpretAllPath)
        .then((f) => {
            interpret = f['interpret'];
            interpretObject = f['interpretObject'];
        })
        .then(() => Promise.all([
            interpret('run selenium'),
            // add some extra services
            interpret('linux dev tools'),
            interpret('vnc html'),
            interpret('vnc docker'),
        ]))
        .then(r => interpretObject(r.map(r => r[0])))
        .then(r => {
            // lets fix the middle result
            r[1].code = bashToRun(r[1].code);
            r[2].code = bashToRun(r[2].code);
            return r.map(r => r.code).join('\n');
        })
        .then(r => {
            // save the Dockerfile
            writeFileSync(outputFile, r);
            return r;
        });
};
(seleniumDocker);


[Function: seleniumDocker]

How to set up selenium server?

In [2]:
var path = require('path');
var execSync = require('child_process').execSync;
var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');
var interpretAllPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');
var interpret, interpretObject, execCmd;
var getSeleniumServer = (name = 'act-selenium') => {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') { throw err; }
    }
    var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));
    return seleniumDocker(DOCKERFILE)
        .then(() => importer.importNotebook(interpretAllPath))
        .then((f) => {
            interpret = f['interpret'];
            interpretObject = f['interpretObject'];
        })
        .then(() => Promise.all([
            interpret('spawn child process')
        ]))
        .then(r => interpretObject(r.map(r => r[0])))
        .then(r => {
            execCmd = eval("'use strict';" + r[0].code);
            return execCmd('docker ps -a');
        }).then(r => {
            var build = 'docker build -t ' + name + ' "' 
                + path.dirname(DOCKERFILE) + '"\n'
                + (r[0].indexOf(name) > -1 
                   ? ('docker stop ' + name + '\n') : '')
                + 'docker run --rm --shm-size=2g -d '
                + '--name ' + name + ' ' 
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads ' 
                + name + '\n'
                + 'sleep 5\n';
            return execCmd(build)
        });
};
(getSeleniumServer);

[Function: getSeleniumServer]

In [3]:
$$.async();
try {
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));
} catch (e) {
    $$.sendError(e);
}


Already imported
[ 'How to import a notebook.ipynb',
  'How to interpret all notebooks.ipynb' ]
[Function: execCmd]
[Function: execCmd]


[ 'Sending build context to Docker daemon   59.9kB\r\r\nStep 1/22 : FROM selenium/standalone-chrome-debug\n ---> 593e47f379db\nStep 2/22 : EXPOSE 4200\n ---> Using cache\n ---> cf96a9aac1f7\nStep 3/22 : EXPOSE 4444\n ---> Using cache\n ---> 3945a29b8f5b\nStep 4/22 : EXPOSE 3000\n ---> Using cache\n ---> b8b75fe784a3\nStep 5/22 : ENV CHROME_USER_DATA_DIR /usr/profile\n ---> Using cache\n ---> 00409baf3489\nStep 6/22 : WORKDIR /home/seluser\n ---> Using cache\n ---> b8a12d57779f\nStep 7/22 : USER root\n ---> Using cache\n ---> aaa0d8320a97\nStep 8/22 : RUN mkdir /usr/profile\n ---> Using cache\n ---> 62eef137b4b0\nStep 9/22 : RUN mkdir /usr/downloads\n ---> Using cache\n ---> bf11798fa872\nStep 10/22 : RUN chown seluser:seluser -R /usr/profile\n ---> Using cache\n ---> 6cd96f04f28a\nStep 11/22 : RUN chown seluser:seluser -R /usr/downloads\n ---> Using cache\n ---> 52b7a641fd74\nStep 12/22 : RUN chmod 777 -R /usr/profile\n ---> Using cache\n ---> 0b526765e8e3\nStep 13/22 : RUN chmod 777 -

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [4]:
var vncIframe = () => `
<div style="display:block; width:100%; padding-bottom:75%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="http://localhost:6080/vnc.html?password=secret&amp;host=localhost&amp;port=6080&amp;autoconnect=true&amp;resize=downscale&amp;view_only=true&amp;reconnect=true"></iframe></div>
`;
(vncIframe);

[Function: vncIframe]

In [5]:
$$.mime({'text/html': vncIframe()});

## 3

How to run a selenium test on the Docker machine?
